## Installing & importing packages

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install tflearn

In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [0]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
import pickle

## Data parsing - cleaning & One Hot Encoding



In [0]:
with open('/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/data/intents.json') as file:
  data = json.load(file)

In [21]:
nltk.download('punkt')
try:
    with open("/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/data/data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
        print('Pickle Loaded...')
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = np.array(training)
    output = np.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Pickle Loaded...


## Training Model

In [23]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
# model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
# model.save("model.tflearn")

try:
    model.load("/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/training/model.tflearn")
    print('Model Loaded....')
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 4999  | total loss: 0.08464 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.08464 - acc: 0.9991 -- iter: 32/34
Training Step: 5000  | total loss: 0.08444 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.08444 - acc: 0.9992 -- iter: 34/34
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Bot init

In [0]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)


def chat():
    print('Now you are talking with GB (GOOD BOI)\n(type "--quit" to stop)')
    while True:
        inp = input("You: ")
        if inp.lower() == "--quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

## Let's chat with the bot

In [35]:
chat()

Now you are talking with GB (GOOD BOI)
(type "--quit" to stop)
You: ki khobor bhai?
bhalo achi
You: apnar boyos koto bhai?
I am just a bot
You: what is your name?
I'm GOOD BOI!
You: ki ache apnar dokan e bhai?
Lal shirt ache :) Niben?
You: kokhon dokan khulen?
We are open 7am-4pm Monday-Friday!
You: what do you recommend
Lal shirt ache :) Niben?
You: jai bhai. pore kotha hobe.
Talk to you later
You: --quit
